In [30]:
%load_ext autoreload
%autoreload 2
%pylab inline

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
%pylab is deprecated, use %matplotlib inline and import the required libraries.
Populating the interactive namespace from numpy and matplotlib


In [31]:
import sys
import glob
import pandas as pd
import os
import seaborn as sns
from tqdm import tqdm_notebook as tqdm
from statsmodels.distributions.empirical_distribution import ECDF
from collections import defaultdict
import open_spiel.python.examples.ubc_dispatch as dispatch
import os
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
from sklearn.model_selection import ParameterGrid
from pathlib import Path
import torch.nn.functional as F
import torch

In [32]:
game_dir = os.environ["CLOCK_AUCTION_CONFIG_DIR"]

# For running the real dumb test case

In [33]:
PREFIX = 'jun5'
sats_games = [f'{PREFIX}/' + os.path.basename(x).replace('.json','') for x in glob.glob(f'{game_dir}/{PREFIX}/*')]
sats_games

['jun5/jun5_1_high_speed',
 'jun5/jun5_2_hide_demand',
 'jun5/jun5_3_high_speed',
 'jun5/jun5_3_hide_demand',
 'jun5/jun5_4_high_speed',
 'jun5/jun5_0_base',
 'jun5/jun5_4_hide_demand',
 'jun5/jun5_0_hide_demand',
 'jun5/jun5_1_hide_demand',
 'jun5/jun5_4_base',
 'jun5/jun5_2_base',
 'jun5/jun5_3_base',
 'jun5/jun5_0_high_speed',
 'jun5/jun5_1_base',
 'jun5/jun5_2_high_speed']

In [34]:
pgrid = ParameterGrid([
    {
        # Defaults
        'game': sats_games
        # 'game': [
            # 'small_sats_complete_symmetric',
            # 'very_small_sats_complete_symmetric'
        # ], 
    },
])
print(len(pgrid))

15


In [35]:
for config in pgrid:
    CONFIG = 'ppo_may31_23' # The YML network config folder for the main jobs
    BR_CONFIG = 'ppo_may31_23' # Ditto for BRs
    N_SEEDS = 3 # Number of seeds of each experiment to launch
    extra_name = ''
    overrides = f'--overwrite_db True --br_portfolio_path {BR_CONFIG} --eval_every 999_000_000 --eval_zero false --total_timesteps 5_000_000 --dispatch_br true --br_overrides "--total_timesteps 1_000_000"'        
    dispatch.dispatch_experiments(CONFIG, base_job_name='jun5ppo', game_name=config['game'], overrides=overrides, n_seeds=N_SEEDS, extra_name=extra_name)

Submitted batch job 82074
Submitted batch job 82075
Submitted batch job 82076
Submitted batch job 82077
Submitted batch job 82078
Submitted batch job 82079
Submitted batch job 82080
Submitted batch job 82081
Submitted batch job 82082
Submitted batch job 82083
Submitted batch job 82084
Submitted batch job 82085
Submitted batch job 82086
Submitted batch job 82087
Submitted batch job 82088
Submitted batch job 82089
Submitted batch job 82090
Submitted batch job 82091
Submitted batch job 82092
Submitted batch job 82093
Submitted batch job 82094
Submitted batch job 82095
Submitted batch job 82096
Submitted batch job 82097
Submitted batch job 82098
Submitted batch job 82099
Submitted batch job 82100
Submitted batch job 82101
Submitted batch job 82102
Submitted batch job 82103
Submitted batch job 82104
Submitted batch job 82105
Submitted batch job 82106
Submitted batch job 82107
Submitted batch job 82108
Submitted batch job 82109
Submitted batch job 82110
Submitted batch job 82111
Submitted ba

# For testing params

In [221]:
pgrid = ParameterGrid([
    {
        # Defaults
        'game': ['jan12_confused_national_base'], 
    },
])
print(len(pgrid))

1


In [ ]:
for config in pgrid:
    CONFIG = 'jan17' # The YML network config folder for the main jobs
    BR_CONFIG = 'jan17' # Ditto for BRs
    N_SEEDS = 1 # Number of seeds of each experiment to launch
    overrides = f'--overwrite_db True --br_portfolio_path {BR_CONFIG} --eval_every 5_000_000 --eval_zero false --total_timesteps 5_000_000 --dispatch_br false --br_overrides "--total_timesteps 1_000_000"'        
    dispatch.dispatch_experiments(CONFIG, base_job_name='jan17_many_configs_2', game_name=config['game'], overrides=overrides, n_seeds=N_SEEDS, extra_name=extra_name)


# For running experiments

In [45]:
pgrid = ParameterGrid([
    {
        # Defaults
        'game': [Path(x).stem for x in glob.glob(f'{game_dir}/jan12*') if 'base' in x], 
    },
    # {   # Potentials
        # 'game': [Path(x).stem for x in glob.glob(f'{game_dir}/jan12*')], 
    #     'potential_function': ['auction_length', 'revenue', 'pricing'],
    #     'scale_coef': [1, 5],
    # },
])
print(len(pgrid))

5


In [49]:
pgrid = ParameterGrid([
    {
        # Defaults
        'game': ['jan12_confused_national_base', 'jan12_regional_vs_national_base'], 
    },
    {   # Potentials
        'game': ['jan12_confused_national_base', 'jan12_regional_vs_national_base'], 
        'potential_function': ['neg_auction_length'],
        'scale_coef': [5, 20],
    },
])

In [50]:
print(list(pgrid))

[{'game': 'jan12_confused_national_base'}, {'game': 'jan12_regional_vs_national_base'}, {'game': 'jan12_confused_national_base', 'potential_function': 'neg_auction_length', 'scale_coef': 5}, {'game': 'jan12_confused_national_base', 'potential_function': 'neg_auction_length', 'scale_coef': 20}, {'game': 'jan12_regional_vs_national_base', 'potential_function': 'neg_auction_length', 'scale_coef': 5}, {'game': 'jan12_regional_vs_national_base', 'potential_function': 'neg_auction_length', 'scale_coef': 20}]


In [55]:
for config in pgrid:
    CONFIG = 'feb28' # The YML network config folder for the main jobs
    BR_CONFIG = 'feb28' # Ditto for BRs
    N_SEEDS = 1 # Number of seeds of each experiment to launch
    overrides = f'--overwrite_db True --eval_every 10_000_000 --eval_zero false --total_timesteps 10_000_000'
    potential_function = config.get('potential_function')
    scale_coef = config.get('scale_coef')
    extra_name = ''
    if potential_function is not None:
        overrides += f' --potential_function {potential_function}'
        extra_name += f'_pot{potential_function}_'
    
    if scale_coef is not None:
        overrides += f' --scale_coef {scale_coef}'
        extra_name += f'_s_{scale_coef}_'
        
    dispatch.dispatch_experiments(CONFIG, base_job_name='feb28', game_name=config['game'], overrides=overrides, n_seeds=N_SEEDS, extra_name=extra_name)


Submitted batch job 81282
Submitted batch job 81283
Submitted batch job 81284
Submitted batch job 81285
Submitted batch job 81286
Submitted batch job 81287
Submitted batch job 81288
Submitted batch job 81289
Submitted batch job 81290
Submitted batch job 81291
Submitted batch job 81292
Submitted batch job 81293


In [ ]:
# for config in pgrid:
#     CONFIG = 'feb28' # The YML network config folder for the main jobs
#     BR_CONFIG = 'feb28' # Ditto for BRs
#     N_SEEDS = 1 # Number of seeds of each experiment to launch
#     overrides = f'--overwrite_db True --br_portfolio_path {BR_CONFIG} --eval_every 10_000_000 --eval_zero false --total_timesteps 10_000_000 --dispatch_br true --br_overrides "--total_timesteps 1_000_000"'
#     potential_function = config.get('potential_function')
#     scale_coef = config.get('scale_coef')
#     extra_name = ''
#     if potential_function is not None:
#         overrides += f' --potential_function {potential_function}'
#         extra_name += f'_pot{potential_function}_'
    
#     if scale_coef is not None:
#         overrides += f' --scale_coef {scale_coef}'
#         extra_name += f'_s_{scale_coef}_'
        
#     dispatch.dispatch_experiments(CONFIG, base_job_name='jan22', game_name=config['game'], overrides=overrides, n_seeds=N_SEEDS, extra_name=extra_name)


In [ ]:
sats_games = [os.path.basename(x).replace('.json','') for x in glob.glob(f'{game_dir}/sats*')]
# sats_games = ['sats_2regions_2licenses_2players_5types']
# sats_games = ['sats_3regions_3licenses_5types']
# sats_games = ['sats_2regions_2licenses_2players_5types', 'sats_3regions_3licenses_5types']
# sats_games = ['rw_game_2_war_speedy_2']
# sats_games = ['confused_national']


# sats_games = ['confused_national', 'confused_national_with_signal', 'confused_national_with_undersell']
# sats_games = ['confused_national_with_signal', 'confused_national_with_undersell']
# sats_games = ['confused_national', 'confused_national_with_undersell']

sats_games = ['confused_national', 'regional_vs_national']

# sats_games = ['regional_vs_national']
# sats_games = ['regional_vs_national_signal']


In [ ]:
# SATS Games
CONFIG = 'dec11trap'
BR_CONFIG = 'dec11trap'
N_SEEDS = 1
# overrides = f'--overwrite_db True --br_portfolio_path {BR_CONFIG} --eval_every 999999 --eval_zero false --total_timesteps 1_000_000 --dispatch_br false'
# NOTE: eval every is phrased in units of UPDATES, not steps
overrides = f'--overwrite_db True --br_portfolio_path {BR_CONFIG} --eval_every 2_500_000 --eval_zero false --total_timesteps 10_000_000 --dispatch_br false --br_overrides "--total_timesteps 2_000_000"'
for GAME_NAME in sats_games:
    dispatch.dispatch_experiments(CONFIG, base_job_name='itsatrap_decorator16', game_name=f'{GAME_NAME}', overrides = overrides, n_seeds=N_SEEDS)
    # dispatch.dispatch_experiments(CONFIG, base_job_name='sats_games_11_with_require_br', game_name=f'{GAME_NAME}', overrides = overrides + ' --iterate_br false --require_br true', n_seeds=N_SEEDS)
    # dispatch.dispatch_experiments(CONFIG, base_job_name='sats_games_11_with_random_ic', game_name=f'{GAME_NAME}', overrides = overrides + ' --random_ic true', n_seeds=N_SEEDS)

In [ ]:
# SATS Games
# CONFIG = 'aug11'
# BR_CONFIG = 'oct31'
CONFIG = 'dec6'
BR_CONFIG = 'dec6br'
N_SEEDS = 3
# overrides = f'--overwrite_db True --br_portfolio_path {BR_CONFIG} --eval_every 999999 --eval_zero false --total_timesteps 1_000_000 --dispatch_br false'
# NOTE: eval every is phrased in units of UPDATES, not steps
overrides = f'--overwrite_db True --br_portfolio_path {BR_CONFIG} --eval_every 2_500_000 --eval_zero false --total_timesteps 10_000_000 --dispatch_br true --br_overrides "--total_timesteps 2_000_000"'
for GAME_NAME in sats_games:
    dispatch.dispatch_experiments(CONFIG, base_job_name='new_configs_dec_8', game_name=f'{GAME_NAME}', overrides = overrides, n_seeds=N_SEEDS)
    # dispatch.dispatch_experiments(CONFIG, base_job_name='sats_games_11_with_require_br', game_name=f'{GAME_NAME}', overrides = overrides + ' --iterate_br false --require_br true', n_seeds=N_SEEDS)
    # dispatch.dispatch_experiments(CONFIG, base_job_name='sats_games_11_with_random_ic', game_name=f'{GAME_NAME}', overrides = overrides + ' --random_ic true', n_seeds=N_SEEDS)

In [2]:
# !squeue --format="%.18i %.65j %.2t %.10M %.6D %R" | grep "external" | awk '{print $1}' | xargs scancel

In [3]:
!squeue --format="%.18i %.90j %.2t .%.10M %.6D %R"

             JOBID                                                                                       NAME ST .      TIME  NODES NODELIST(REASON)
             86619                       jun19_jun19_0_medium_speed-cfr_outcomemccfr_outcome-100_jun21outcome  R .  15:07:34      1 ip-10-255-1-191
             86620                       jun19_jun19_0_medium_speed-cfr_outcomemccfr_outcome-101_jun21outcome  R .  15:07:34      1 ip-10-255-1-191
             86621                       jun19_jun19_0_medium_speed-cfr_outcomemccfr_outcome-102_jun21outcome  R .  15:07:34      1 ip-10-255-1-192
             86626                  jun19_jun19_0_undersell_allowed-cfr_outcomemccfr_outcome-101_jun21outcome  R .  15:07:34      1 ip-10-255-1-156
             86640                               jun19_jun19_2_base-cfr_outcomemccfr_outcome-100_jun21outcome  R .  15:07:34      1 ip-10-255-1-176
             86641                               jun19_jun19_2_base-cfr_outcomemccfr_outcome-101_jun21outcome  

In [100]:
!squeue | wc -l

33


In [88]:
!squeue --format="%.18i %.90j %.2t .%.10M %.6D %R" | grep "modal" | wc -l

23


In [73]:
!scancel -u ubuntu

In [ ]:
# !scancel  46891 46893

In [ ]:
!rm -rf /shared/outputs/minitest/
try:
    Experiment.objects.get(name='minitest').delete()
except:
    pass
dispatch.dispatch_experiments('feb2', base_job_name='minitest', game_name='tiny', overrides = '--br_portfolio_path feb2 --eval_every 500 --num_training_episodes 1_000 --br_overrides "--num_training_episodes 1_000" --eval_overrides "--num_samples 100"')

In [ ]:
BestResponse.objects.last()

In [ ]:
Experiment.objects.all()

In [ ]:
!sacct --starttime 2022-03-11 --format=User,JobID,Jobname,partition,state,time,start,end,elapsed,MaxRss,MaxVMSize,nnodes,ncpus,nodelist

In [ ]:
from open_spiel.python.examples.ubc_plotting_utils import parse_run
from auctions.webutils import *

def find_best_checkpoint(run, max_t=None):
    ev_df = parse_run(run, max_t)
    best_t = ev_df.groupby('t')['ApproxNashConv'].first().idxmin()
    nash_conv_by_t = ev_df.groupby('t')['ApproxNashConv'].first()
    best_checkpoint = get_checkpoint(run, t=best_t)
    return nash_conv_by_t, best_checkpoint, nash_conv_by_t.min()


run = EquilibriumSolverRun.objects.last()
find_best_checkpoint(run)

In [ ]:
Experiment.objects.get(name='large_game_2_no_activity.json').delete()

In [ ]:
Game.objects.get(name='large_game_2_pricing_extreme.json').delete()

In [ ]:
!python /apps/open_spiel/web/auctions/manage.py nfsp --seed 109 --filename rounds.json --network_config_file mar17/lstm --experiment_name lstmtest --job_name "lstm_for_greg" --dispatch_br false --br_portfolio_path feb22 --eval_every 10_000 --num_training_episodes 10_000_000 --br_overrides "--num_training_episodes 1_000_000" --report_freq 10_000 --overwrite_db true

In [ ]:
list(Experiment.objects.all().values_list('name', flat=True))

In [ ]:
r = EquilibriumSolverRun.objects.get(name='large_game_3b_hide-mar17lstm-100')

In [ ]:
from auctions.webutils import *
from open_spiel.python.examples.ubc_plotting_utils import *
parse_run(r, conservative=False)

In [ ]:
BREvaluation.objects.filter(best_response__checkpoint__equilibrium_solver_run=r)

In [ ]:
list(BestResponse.objects.filter(checkpoint__equilibrium_solver_run=r).values('name'))

In [ ]:
BestResponse.objects.filter(checkpoint=pk)[0].brevaluation.__dict__

In [ ]:
from open_spiel.python.examples.ubc_plotting_utils import *
parse_run(r, conservative=False)

In [ ]:
BREvaluation.objects.filter(best_response__checkpoint__equilibrium_solver_run=r).values('best_response__name')

In [ ]:
BestResponse.objects.filter(checkpoint__equilibrium_solver_run=r).values('name')

In [ ]:
experiments = ['mar24', 'mar22_4']
all_scripts = []
for e in experiments:
    scripts = glob.glob(f'/shared/outputs/{e}/*/evaluations/*.sh')
    scripts = [s for s in scripts if 'straightforward' not in s]
    scripts = [s for s in scripts if s[-5:] == "_2.sh"]
    all_scripts += scripts

In [ ]:
for script in all_scripts:
    parent = Path(script).parent
    !cd {parent} && sbatch {script}

In [ ]:
x = Evaluation.objects.last()

In [ ]:
x.samples['allocations']

In [ ]:
Evaluation.objects.last()

In [ ]:
python /apps/open_spiel/web/auctions/manage.py nfsp --seed 100 --filename sats_5.json --network_config_file flatmlp/mlp --experiment_name test_require_br --job_name "sats_5_test-flatmlpmlp-100" --dispatch_br false --dry_run true --overwrite_db True --br_portfolio_path flatmlp --eval_every 1_000_000 --eval_zero false --num_training_episodes 8_000_000 --br_overrides "--num_training_episodes 1_000_000" --iterate_br false --require_br true

# Get env_and_model


# run_nfsp(env_and_model, 10_000, True, False, NullResultSaver(), 1, False, NullDispatcher(), eval_every, eval_every_early, eval_exactly, eval_zero, report_freq, dispatch_br, agent_selector, random_ic=False):

In [ ]:
BREvaluation.objects.filter(best_response__name='straightforward').last().expected_value_cdf

In [ ]:
!pip install wandb

In [ ]:
Evaluation.objects.last().samples.keys()

In [ ]:
Evaluation.objects.filter(
                    checkpoint__equilibrium_solver_run__experiment__name='testevolution3',
                    checkpoint__equilibrium_solver_run__generation=0,
                ).count()

In [ ]:
Evaluation.objects.last()

In [ ]:
!python /apps/cluster_open_spiel/web/auctions/manage.py ppo --filename bully1_2.json --network_config_file aug11/ppo --experiment_name bully1 --job_name "baseline" --overwrite_db True --eval_every 9999 --eval_zero false --total_timesteps 1_000_000 --use_wandb true --dry_run true

In [ ]:
!python /apps/cluster_open_spiel/web/auctions/manage.py ppo --filename bully_2.json --network_config_file aug11/ppo_potential_neg_pricing --experiment_name bully --job_name "neg_pricing" --overwrite_db True --eval_every 9999 --eval_zero false --total_timesteps 1_000_000 --use_wandb true --dry_run false

In [ ]:
Evaluation.objects.last()

In [ ]:
BestResponse.objects.exclude(name__contains='straightforward').last()

In [ ]:
potential_functions = ('auction_length', 'neg_auction_length', 'neg_revenue', 'revenue')

In [ ]:
from pathlib import Path
import glob
import yaml
ROOT = 'configs/nov23'
for f in glob.glob(f'{ROOT}/*.yml'):
    config = yaml.safe_load(open(f, 'rb'))
    stem = Path(f).stem
    for potential in potential_functions:
        config['potential_function'] = potential
        yaml.dump(config, open(f'{ROOT}/{stem}_{potential}.yml', 'w'))

In [ ]:
Game.objects.filter(name__startswith='dec15').delete()

# Test CFR

In [44]:
!python /apps/cluster_open_spiel/open_spiel/python/examples/ubc_mccfr_cpp_example.py --filename small_sats_complete_symmetric.json --iterations 1000 

Reading from env variable CLOCK_AUCTION_CONFIG_DIR. If it is not set, there will be trouble.
CLOCK_AUCTION_CONFIG_DIR=/apps/open_spiel/configs
Parsing configuration from /apps/open_spiel/configs/small_sats_complete_symmetric.json
Done config parsing
I0204 03:36:16.852158 140691547054208 ubc_mccfr_cpp_example.py:169] Using C++ implementations
I0204 03:36:16.852356 140691547054208 ubc_mccfr_cpp_example.py:171] Using CFR solver


In [11]:
from cachetools.keys import hashkey
hashkey(())

((),)

In [13]:
hashkey((72,3))

((72, 3),)